In [4]:
import datetime
import time
import json

from dateutil import tz
from timezonefinder import TimezoneFinder

import astropy
import astropy.coordinates as coord
import astropy.units as u

from astropy.coordinates import get_sun, AltAz, EarthLocation
from astropy.time import Time
from astropy.utils import iers

In [6]:
# the file used in astropy failed to download. I used the following mirror:
iers.Conf.iers_auto_url.set('ftp://cddis.gsfc.nasa.gov/pub/products/iers/finals2000A.all')

In [8]:
data = {
    'lat': 49.2699648,
    'long': -123.1290368,
    'epoch': 1588704959.321,
    'orientation': -10.2
}

In [9]:
class TimeZoneUtils():
    def __init__(self):
        self.tz_finder = TimezoneFinder()
        
    # get the time zone using lat and long
    def get_time_zone(self, lat, long):
        return self.tz_finder.timezone_at(lat=lat, lng=long)    

In [13]:
class DateUtils():
    def __init__(self, tz_utils):
        self.tz_utils = tz_utils
        pass

    # get the UTC datetime
    def get_utc_datetime(self, epoch):
        utc_date = datetime.datetime.utcfromtimestamp(epoch)
        return utc_date

    
    # convert UCT to local datetime
    def get_local_datetime(self, epoch, lat, long):
        utc_datetime = self.get_utc_datetime(epoch)
        
        timezone_from = tz.gettz('UTC')
        timezone_to = tz.gettz(self.tz_utils.get_time_zone(lat, long))
        
        utc_datetime = utc_datetime.replace(tzinfo=timezone_from)
        local_datetime = utc_datetime.astimezone(timezone_to)
        
        local_datetime = local_datetime.strftime('%Y-%m-%d %H:%M:%S.%f')
        return datetime.datetime.strptime(local_datetime, '%Y-%m-%d %H:%M:%S.%f')

In [14]:
tz_utils = TimeZoneUtils()
date_utils = DateUtils(tz_utils)

In [20]:
utc_datetime = date_utils.get_utc_datetime(data['epoch'])
local_datetime = date_utils.get_local_datetime(data['epoch'], data['lat'], data['long'])
local_timezone = tz_utils.get_time_zone(data['lat'], data['long'])

print('Picture is taken at %s - UTC time zone.' % utc_datetime)
print('Picture is taken at %s - %s time zone.' % (local_datetime, local_timezone))

Picture is taken at 2020-05-05 18:55:59.321000 - UTC time zone.
Picture is taken at 2020-05-05 11:55:59.321000 - America/Vancouver time zone.


In [21]:
AZIMUTHAL_DIFF_BOUNDARY = 30
SUN_ALTITUDE_BOUNDARY = 45

In [22]:
class CoordinationUtils():
    def __init__(self, date_utils, lat, long, epoch, orientation):
        self.data_utils = date_utils

        self.pic_location = coord.EarthLocation(lat=lat * u.deg, lon=long * u.deg)
        self.pic_datetime = Time(date_utils.get_utc_datetime(epoch))
        
        self.azimuth = 0 
        self.altitude = 0 
        self.orientation = orientation
        
        self.sun = coord.get_sun(self.pic_datetime)
    
    def __compute_azimuth_and_altitude(self):
        alt_and_az = coord.AltAz(location=self.pic_location, obstime=self.pic_datetime)
        self.altitude = self.sun.transform_to(alt_and_az).alt.degree
        self.azimuth = self.sun.transform_to(alt_and_az).az.degree

    def detect_glare(self):
        self.__compute_azimuth_and_altitude()
        
        temp_orientation = (self.orientation + 360) % 360

        phi = abs(self.azimuth - temp_orientation) % 360
        if (phi > 180):
            azimuth_distance = 360 - phi
        else:
            azimuth_distance = phi
        
        if (azimuth_distance <= AZIMUTHAL_DIFF_BOUNDARY and self.altitude <= SUN_ALTITUDE_BOUNDARY):
            return True

        return False
    
    def print_glare_data(self):
        print('              azimuth:', azimuth)
        print('          orientation:', orientation)
        print('             altitude:', altitude)
        print('azimuth - orientation:', azimuth_distance)
        print('             altitude:', altitude)

    def print_picture_data(self):
        print('Picture location:', self.pic_location)
        print('    Picture time:', self.pic_datetime)
        print('             Sun:', self.sun)

In [23]:
coord_utils = CoordinationUtils(date_utils, data['lat'], data['long'], data['epoch'], data['orientation'])

In [24]:
glare = json.dumps({'glare': coord_utils.detect_glare()})
glare

'{"glare": false}'